In [1]:
import json
import requests
import requests
import traceback
import time

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import exists

In [3]:
# TODO: Add retry logic to this in case of hangups.
def run_query(query, retries=10):
    """
    Query graphQL API.

    If timeerror
    """
    headers = {"X-API-KEY": "BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4"}

    retries_counter = 0
    try:
        request = requests.post(
            "https://graphql.bitquery.io/", json={"query": query}, headers=headers
        )
        result = request.json()
        # print(dir(request.content))
        # Make sure that there is no error message
        # assert not request.content.errors
        assert "errors" not in result
    except:
        while (
            (request.status_code != 200
            or "errors" in result)
            and retries_counter < 10
        ):
            print(datetime.now(), f"Retry number {retries_counter}")
            if "errors" in result:
                print(result["errors"])
            print(datetime.now(), f"Query failed for reason: {request.reason}. sleeping for {150*retries_counter} seconds and retrying...")
            time.sleep(150*retries_counter)
            request = requests.post(
                "https://graphql.bitquery.io/",
                json={"query": query},
                headers=headers,
            )
            retries_counter += 1
        if retries_counter >= retries:
            raise Exception(
                "Query failed after {} retries and return code is {}.{}".format(
                    retries_counter, request.status_code, query
                )
            )
    return request.json()

In [4]:
query = """
    query MyQuery {
          solana {
            instructions(
              time: {between: ["2022-07-01", "2022-07-07"]}
              success: {is: true}
              programId: {is: "3HUeooitcfKX1TSCx2xEpg2W31n6Qfmizu7nnbaEWYzs"}
              options: {limit: 8700}
            ) {
              block {
                timestamp {
                  iso8601
                }
              }
              transaction {
                signature
                feePayer
              }
              data {
                base58
              }
              log {
                instruction
                result
                logs
              }              
              
            }
          }
        }
"""

In [5]:
result = run_query(query)

In [6]:
df = pd.json_normalize(result["data"]["solana"]["instructions"])


In [7]:
df["log.instruction"].unique()

array(['CalculateCurrentYield', 'CalculationAgent',
       'OverrideObservationPeriod', 'DepositVault', 'WithdrawVault'],
      dtype=object)

In [8]:
df.loc[df["log.instruction"]=="EndRound"]

,block.timestamp.iso8601,transaction.signature,transaction.feePayer,data.base58,log.instruction,log.result,log.logs
